# Supply Chain Dataset Analysis---<B> Prompt For Chat GPT </B> <br>Hello, I am trying to create a python code which will help me generate data. I am looking to generate data for Supply Chain. This is a data for a warehouse which has information of orders that is incoming every day. These are the values I am looking for and I have also provided the information each variable should have:* order_id – unique ID for each order (random numerical). This should be unique for each item.* item_category – Electronics, Apparel, Office Supplies, Food & Beverage, Medical Supplies.* supplier_name – Supplier_A, Supplier_B, Supplier_C.* Warehouse_name - Name of Warehouse (create random name of Business name for should have characters a-z, A-Z and numerical values 1-9). This doesn’t have to be unique and each warehouse has a location it is associated with.* warehouse_location – Generate Random location for each unique warehouse should be associated with a warehouse_name.* fc_associate – ID or name of associate who processed the order.* order_status – Pending, Received, Backordered, Canceled.* total_quantity_required – how many units were ordered.* total_quantity_received – how many units actually arrived.* total_quantity_in_transfer – units still pending shipment (required – received).* unit_price – price per unit (varies by category).* order_cost – total_quantity_received × unit_price.* lead_time_days – how many days from order to delivery. (make it random so, I want a couple of data where need by date > received date and also a couple of times where received date is null).* need_by_date – deadline for when the order is needed. (I want a couple of data where need by date > received date and have some need by date > today and some less than today with receive_date as null).* receive_date – when the items were actually received. (make it random so, I want a couple of data where need by date > received date and also a couple of times where received date is null).---

<B> Data Generation Code for Supply Chain </B>

In [ ]:
import pandas as pdimport numpy as npimport randomfrom faker import Faker# Initializefake = Faker()np.random.seed(42)random.seed(42)# Parametersn = 2000  # number of rowsitem_categories = {    "Electronics": (50, 500),    "Apparel": (10, 100),    "Office Supplies": (5, 50),    "Food & Beverage": (2, 20),    "Medical Supplies": (20, 200)}suppliers = ["Supplier_A", "Supplier_B", "Supplier_C"]statuses = ["Pending", "Received", "Backordered", "Canceled"]# Generate warehouse names (ensuring repetition)warehouses = [f"WH_{fake.lexify(text='???').upper()}{random.randint(1,9)}" for _ in range(10)]warehouse_locs = {w: fake.city() for w in warehouses}data = []for i in range(n):    order_id = fake.unique.random_int(10000, 99999)    category = random.choice(list(item_categories.keys()))    supplier = random.choice(suppliers)    warehouse = random.choice(warehouses)    warehouse_loc = warehouse_locs[warehouse]    fc_associate = f"Assoc_{random.randint(100,999)}"        status = random.choice(statuses)        qty_required = random.randint(10, 200)    qty_received = qty_required if status == "Received" else random.randint(0, qty_required)    qty_in_transfer = qty_required - qty_received        unit_price = random.randint(*item_categories[category])    order_cost = qty_received * unit_price        # Dates    order_date = fake.date_between(start_date="-60d", end_date="today")    need_by_date = order_date + pd.Timedelta(days=random.randint(3, 15))        if status == "Received":        lead_time = random.randint(1, 20)        receive_date = order_date + pd.Timedelta(days=lead_time)        # enforce ~70% late deliveries        if random.random() < 0.7:            receive_date = need_by_date + pd.Timedelta(days=random.randint(1, 7))    else:        lead_time = None        receive_date = None        data.append([        order_id, category, supplier, warehouse, warehouse_loc, fc_associate,        status, qty_required, qty_received, qty_in_transfer,        unit_price, order_cost, lead_time, need_by_date, receive_date    ])# Build DataFramecolumns = [    "order_id","item_category","supplier_name","warehouse_name","warehouse_location",    "fc_associate","order_status","total_quantity_required","total_quantity_received",    "total_quantity_in_transfer","unit_price","order_cost","lead_time_days",    "need_by_date","receive_date"]df = pd.DataFrame(data, columns=columns)# Export to CSVdf.to_csv("warehouse_orders.csv", index=False)print("Generated 2000 rows and saved to warehouse_orders.csv")print(df.head())

--<B> Understanding Data </B>* I wanted to check the starting and ending rows of the data so I added the following lines of code. This is so I can check what type of data were generated.

In [ ]:
# I wanted to check the starting and ending rows of the data so I added the following lines of code.print("These are the starting 10 rows for the data \n",df.head(10))print("These are the ending 5 rows for the data \n",df.tail(10))

Similarly I wanted to check : * I want to check the dimensions of the DataFrame so df.shape (Output : 2000 rows and 15 columns)* Second I wanted to check the column names of the DataFrame and the number of column names (Output : It has listed all the 15 column names)* Third, I wanted to check the data types of each column in the DataFrame (Output : It has listed all the 15 column names with their respective data types)* I also wanted to check the total number of elements in the DataFrame (Output : 30000 which is 2000*15)* I also wanted to check the number of dimensions of the DataFrame (Output : 2 which means it is a 2D array)* Finally, I thought to get a summary of the DataFrame, including non-null counts and memory usage.--My starting point of my analysis was to check if there are any warehouses that are at risk where we are waiting for pending items and that are overdue. So, to start with this analysis I started with checking the unique warehouse and used describe methods to get a summary of the 'warehouse_name' column, including count, unique values, top value, and frequency.

In [ ]:
# I want to understand the data better so I added the following lines of code.df.shape # To check the dimensions of the DataFrame (Output : 2000 rows and 15 columns)df.columns # To check the column names of the DataFrame (Output : It has listed all the 15 column names)df.dtypes # To check the data types of each column in the DataFrame (Output : It has listed all the 15 column names with their respective data types)df.size # To check the total number of elements in the DataFrame (Output : 30000 which is 2000*15)df.ndim # To check the number of dimensions of the DataFrame (Output : 2 which means it is a 2D array)df.info() # To get a concise summary of the DataFrame, including non-null counts and memory usagedf['warehouse_name'].unique() # To get the unique values in the 'warehouse_name' columndf['warehouse_name'].describe() # To get a summary of the 'warehouse_name' column, including count, unique values, top value, and frequency

Once I have the information of the warehouse, I wanted to check how many orders were placed for each warehouse. Once this was completed, I wanted to check which warehouse is having the most lead time meaning which warehouse is unable to receive the items on time. As items not being delivered on time can cause delay in the projects and impact. This analysis can help initially find the warehouse which is having issues with the items not being delivered on time. So, I used pandas to find the min, max, mean lead time with the total order and total purchase order.

In [ ]:
# To check how many orders were placed from each warehouse df['warehouse_name'].value_counts()# I also wanted the location of each warehouse so I added the following line of code. df.groupby(['warehouse_name', 'warehouse_location']).size().reset_index(name='count')# To check the average lead time days for each warehouse. To get information about how efficient each warehouse is in terms of lead time.df.groupby(['warehouse_name', 'warehouse_location'])['lead_time_days'].mean().round(2).reset_index()df.groupby(['warehouse_name', 'warehouse_location'])['lead_time_days'].mean().round(2).reset_index()# .agg() - I wanted to find the max and min lead time days so it will help me with analyzing the warehouse efficiently.df.groupby(['warehouse_name', 'warehouse_location'])['lead_time_days'].agg(['min','max','mean']).round(3)warehouse_summary = df.groupby(['warehouse_name', 'warehouse_location']).agg(min_lead_time_delay=('lead_time_days', 'min'),max_lead_time_delay=('lead_time_days', 'max'),                                                                             mean_lead_time_delay=('lead_time_days', 'mean'),                                                     total_orders=('order_id', 'count'), total_purchase_amount=('order_cost', 'sum')).round(3).reset_index()warehouse_summary

Once these values were created, I sorted the values in ascending for mean lead delay so I can easily find the highest lead time delays. Then I wanted to check the overdue items so I wrote a query to check on the overdue items. These items are still pending and have not been received yet.

In [ ]:
# Now I want to sort by the total orders and mean lead time for order so that I can see which warehouse has the most orders and how efficient they are in terms of lead time.warehouse_summary.sort_values(by=['mean_lead_time_delay'], ascending=[False],ignore_index=True,inplace=True)# Going back to df for more analysisimport datetimetoday = datetime.date.today()  # gets today's date at midnightdf['order_status'].value_counts() # To see the distribution of different order statuses in the datasetdf.groupby(['warehouse_name', 'order_status']).size()df['is_overdue'] = (    (df['order_status'] == "pending")    & df['receive_date'].isna()    | (df['need_by_date'] < df['receive_date']))df['is_overdue']status_counts = (    df.groupby(['warehouse_name', 'is_overdue'])      .size()      .unstack(fill_value=0)      .reset_index()      .rename(columns={True: 'overdue_orders', False: 'not_overdue_orders'}))warehouse_summary = warehouse_summary.merge(    status_counts,    on='warehouse_name',    how='left')warehouse_summary

Finally, I wanted to find the warehouse with the highest number of orders pending to be delivered and that have the highest lead time so I can find the warehouse which is struggling the most.

In [ ]:
warehouse_summary['not_delivered'] = warehouse_summary['overdue_orders'] / warehouse_summary['total_orders']warehouse_summary['warehouse_delivery_status'] = pd.cut(    warehouse_summary['not_delivered'],       # column to categorize    bins=[-1, 0.10, 0.20, 1],      # define intervals    labels=['Green', 'Yellow', 'Red']  # category labels)warehouse_summary.sort_values(by=['mean_lead_time_delay','warehouse_delivery_status'], ascending=[False,True],ignore_index=True,inplace=True)warehouse_summary